In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

remove_fight_island = False

In [2]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


5


In [3]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20',
  'model_10_23_20',
  'model_10_27_20',
  'model_10_28_20'],
 ['LogisticRegression(max_iter=50000, random_state=75)',
  "DecisionTreeClassifier(max_depth=27, max_leaf_nodes=336, min_samples_leaf=0.1,\n                       random_state=75, splitter='random')",
  "RandomForestClassifier(criterion='entropy', max_depth=10,\n                       min_samples_leaf=0.101, n_estimators=22, n_jobs=-1,\n                       random_state=75)",
  "GradientBoostingClassifier(max_depth=17, max_features='sqrt',\n                           max_leaf_nodes=316, min_samples_leaf=0.1,\n                           n_estimators=195, random_state=75)",
  'GaussianNB(var_smoothing=1e-08)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'Gra

In [4]:
df = pd.read_csv("../data/ufc-master.csv")

In [5]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4533
4533
4533


<ipython-input-5-e95d4bafa98e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 0
<ipython-input-5-e95d4bafa98e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 1


In [6]:
if remove_fight_island:
    ##Let's remove the Fight island contests and see how that affects score
    df_test_no_fight_island = df_test[(df_test['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df_train_no_fight_island = df_train[(df_train['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df_test = df_test_no_fight_island
    df_train = df_train_no_fight_island



print(len(df_test))
print(len(df_train))


200
4533


In [7]:
#Set a value for the nulls in the ranks

weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [8]:
len(models[0])

15

In [9]:
score_list = []
for model_num in range(len(models[0])):
    test_model_name = models[0][model_num]
    test_model = eval(models[1][model_num])
    test_model_features = eval(models[2][model_num])
    test_model_ev = eval(models[3][model_num])
    
    score_list.append(evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True))

(4533, 5)

(4533,)

(4533, 2)

(200, 5)

(200,)

...

(198, 26)

(198,)

(198, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-118,-106,0.598315,0.401685,0
1,140,-177,0.351949,0.648051,1
2,-150,120,0.592936,0.407064,0
3,105,-134,0.507419,0.492581,0
4,-177,140,0.580359,0.419641,1
...,...,...,...,...,...
193,-155,125,0.568679,0.431321,0
194,-175,135,0.640672,0.359328,1
195,-530,375,0.681813,0.318187,0
196,170,-215,0.400522,0.599478,1



          Number of matches: 198
          Number of bets: 132
          Number of winning bets: 56
          Number of losing bets: 77
          Number of underdog bets: 91
          Number of underdog wins: 34
          Number of underdog losses: 57
          Number of Favorite bets: 41
          Number of favorite wins: 21
          Number of favorite losses: 20
          Number of even bets: 1
          Number of even wins: 1
          Number of even losses: 0
          Profit: -1129.8069204547003
          Profit per bet: -8.559143336778032
          Profit per match: -5.706095557852022
          
          


In [10]:
print(models[4])

['173.1684847', '-208.59182319063598', '-827.0117854538566', '662.3292884444202', '-1162.4687784517437', '1205.652357', '385.0625139', '990.0325213', '-469.6786711', '-372.6349433', '-583.4522165', '-11.15784433', '233.6262205', '-690.9821024', '-16.7945354']


In [11]:
print(score_list)

[179.2841997796204, -338.59182319063575, -1338.3073379483208, 755.499978736892, -955.3756943794973, 1165.977715754501, 528.6527731135301, 1500.9399404095232, 493.164776463737, -431.4817088774101, -35.230378360601506, -19.43318130992982, 290.91041589744685, -91.06290010127861, -1129.8069204547003]


In [12]:
models[4] = score_list

In [13]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20',
  'model_10_23_20',
  'model_10_27_20',
  'model_10_28_20'],
 ['LogisticRegression(max_iter=50000, random_state=75)',
  "DecisionTreeClassifier(max_depth=27, max_leaf_nodes=336, min_samples_leaf=0.1,\n                       random_state=75, splitter='random')",
  "RandomForestClassifier(criterion='entropy', max_depth=10,\n                       min_samples_leaf=0.101, n_estimators=22, n_jobs=-1,\n                       random_state=75)",
  "GradientBoostingClassifier(max_depth=17, max_features='sqrt',\n                           max_leaf_nodes=316, min_samples_leaf=0.1,\n                           n_estimators=195, random_state=75)",
  'GaussianNB(var_smoothing=1e-08)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'Gra

In [14]:
with open('models.csv', 'w' , newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI
